# SeisBench Testcase Prediction (Detection/Picking)
This prediction test case uses a small amount of data from the GHOST 2022/2023 nodal geophone acquisition on downstream Thwaites Glacier, West Antarctica.

Waveform data are locally hosted by the script author and available upon request from the   
(co-)principal investigators during the restricted-use period for the 2022/2023 Y2 network.  
After this period data will be accessible via the IRIS Data Management Center.  

#### Script Author
Nathan Stevens (ntsteven@uw.edu)

#### (Co-)Principal Investigators
Sridhar Anandakrishnan (sxa17@psu.edu)
Lucas Zoet (lzoet@wisc.edu)
Paul Winberry (paul.winberry@gmail.com)
#### Attribution
- Code structure based on discussions with Yiyu Ni (University of Washington)
- The GHOST (Geophysical Habitat Of Subglacial Thwaites) project is funded in conjunction by the NSF (PLR-1738913) and NERC (National Environmental Research Council, United Kingdom)

In [ ]:
# Import modules
from seisbench.models import EQTransformer
from obspy import read, Stream, UTCDateTime
import torch
import matplotlib.pyplot as plt
from glob import glob

In [ ]:
# Load data
flist = glob('./data/GHOST_1210_Example/*')
st = Stream()
for f_ in flist:
    st += read(f_)


print(st)

In [ ]:
# Preprocess data
# Ensure correct data sequence (Z12)
st2 = Stream()
st2 += st.select(channel='??Z')
st2 += st.select(channel='??[1N]')
st2 += st.select(channel='??[2E]')

# Filter data to frequency band of observed basal icequakes
st2 = st2.filter('bandpass',freqmin=10,freqmax=200)

# Time dilate traces to make them "look" like tectonic earthquakes (sits in the band common to EQs used for training)
for tr_ in st2:
    tr_.stats.sampling_rate = 100

print(st2)

In [ ]:
# Initialize ML prediction model
model = EQTransformer.from_pretrained('stead')
# Push job to GPU (can be fussy )
# model.to(torch.device('cuda'))

In [ ]:
# Run annotation prediction for pick probabilities and detection function
ann_st = model.annotate(st2)

In [ ]:
# Run classification to get picks
picks = model.classify(st2,batch=256,P_threshold=1e-5,S_threshold=1e-4)

In [ ]:
# Show pick probabilities/detections alongside traces
st3 = st2.copy();
st3 += ann_st.copy();
st3.plot(equal_scale=False,method='full')